<center><img src="../../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2022, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-03-18 17:42:02  
@LastEditors: Liusen  
@LastEditTime: 2020-03-18 17:42:02   

# Train svm model and save

In [ ]:
from PIL import Image
import os
import sys
import numpy as np
import time
from sklearn import svm
from sklearn.externals import joblib
 
# Get all .png files in the specified path
def get_file_list(path):
    return [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".png")]
 
# Parse the name of the .png image file
def get_img_name_str(imgPath):
    return imgPath.split(os.path.sep)[-1]
 
 
# Convert 28px*28px image data to 1*784 numpy vector
# Convert 28px*28px image data to 1*784 numpy vector
# Parameters: imgFile--image name such as: 1.png
# Return: 1*784 numpy vector
def img2vector(imgFile):
    # print("in img2vector func--para:{}".format(imgFile))
    img = Image.open(imgFile).convert('L')
    img_arr = np.array(img, 'i')  # 28px * 28px grayscale image
    img_normalization = np.round(img_arr / 255)  # Normalize the gray value
    img_arr2 = np.reshape(img_normalization, (1, -1))  # 1 * 784 matrix
    return img_arr2
 
# Read all data in a category and convert it into a matrix
# Parameters:
#    basePath:The basic path where the image data is located
#       MNIST-data/train/
#       MNIST-data/test/
#    cla：classification name
#       0,1,2,...,9
# Returns: all data of a certain category-[number of samples * (image width x image height)] matrix
def read_and_convert(imgFileList):
    dataLabel = []  # Storage class label
    dataNum = len(imgFileList)
    dataMat = np.zeros((dataNum, 784))  # dataNum*784 matrix
    for i in range(dataNum):
        imgNameStr = imgFileList[i]
        imgName = get_img_name_str(imgNameStr)  # Get the number of the current number.png
        # print("imgName: {}".format(imgName))
        classTag = imgNameStr.split(os.path.sep)[-2]
        # classTag = imgName.split(".")[0].split("_")[0]  #Get class label (number)
        #print(classTag)
        #print(imgNameStr)
        dataLabel.append(classTag)
        dataMat[i, :] = img2vector(imgNameStr)
    return dataMat, dataLabel
  
# Read training data
def read_all_data():
    cName = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
    #path = sys.path[1]
    train_data_path = 'MNIST_data/train/0' # os.path.join(path, './MNIST_data/train/0')
    print(train_data_path)
    #train_data_path = "./MNIST_data/train/0"
    print('0')
    flist = get_file_list(train_data_path)
    #print(flist)
    dataMat, dataLabel = read_and_convert(flist)
    for c in cName:
        print(c)
        #train_data_path = os.path.join(path, './MNIST_data/train/') + c
        train_data_path = 'MNIST_data/train/' + c
        flist_ = get_file_list(train_data_path)
        dataMat_, dataLabel_ = read_and_convert(flist_)
        dataMat = np.concatenate((dataMat, dataMat_), axis=0)
        dataLabel = np.concatenate((dataLabel, dataLabel_), axis=0)
    # print(dataMat.shape)
    # print(len(dataLabel))
    return dataMat, dataLabel
 
'''
SVC parameter
svm.SVC(C=1.0,kernel='rbf',degree=3,gamma='auto',coef0=0.0,shrinking=True,probability=False,
tol=0.001,cache_size=200,class_weight=None,verbose=False,max_iter=-1,decision_function_shape='ovr',random_state=None)

C：C-SVC penalty parameter C, the default value is 1.0
The larger C is, the equivalent of punishing the slack variable. It is hoped that the slack variable will be close to 0, that is, the penalty for misclassification will increase, which tends to fully split the training set.
The accuracy is high, but the generalization ability is weak. C value is small, the penalty for misclassification is reduced, fault tolerance is allowed, they are regarded as noise points, and the generalization ability is strong.

kernel:Kernel function, default is rbf, it can be set to‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ 
    0 – linear：u'v
 　 1 – Polynomial：(gamma*u'*v + coef0)^degree
  　2 – RBF function：exp(-gamma|u-v|^2)
  　3 –sigmoid：tanh(gamma*u'*v + coef0)


degree: The dimension of the polynomial poly function, which is 3 by default. It will be ignored when other kernel functions are selected. (Useless)

gamma: Kernel function parameters of ‘rbf’, ‘poly’ and ‘sigmoid’. The default is ‘auto’, then 1/n_features will be selected

coef0: constant term of the kernel function. Useful for ‘poly’ and ‘sigmoid’. (Useless)

probability: whether to use probability estimation, the default is False

shrinking: Whether to use shrinking heuristic method, the default is true

tol: the size of the error value for stopping training, the default is 1e-3

cache_size: The cache size of the core function cache, the default is 200

class_weight: The weight of the category, passed in the form of a dictionary. Set the parameter C of the first category to weight *C (C in C-SVC)

verbose: Allow redundant output.

max_iter: Maximum number of iterations. -1 is unlimited.

decision_function_shape: ‘ovo’, ‘ovr’ or None, default = None3 (select ovr, one-to-many)

random_state: seed value, int value when the data is shuffled
The main adjustment parameters are: C, kernel, degree, gamma, coef0
''' 
# Create a model
def create_svm(dataMat, dataLabel,path,decision='ovr'):
    clf = svm.SVC(C=1.0,kernel='rbf',decision_function_shape=decision)
    rf =clf.fit(dataMat, dataLabel)
    joblib.dump(rf, path)
    return clf

 
if __name__ == '__main__':
    # clf = svm.SVC(decision_function_shape='ovr')
    st = time.clock()
    dataMat, dataLabel = read_all_data()
    #path = sys.path[1]
    #model_path=os.path.join(path,'model\\svm.model')
    model_path = 'model/svm.model'
    create_svm(dataMat, dataLabel, model_path, decision='ovr')
    et = time.clock()
    print("Training spent {:.4f}s.".format((et - st)))

